[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/palette-knife25/candidate-free-te/blob/main/notebooks/CandidateFreeTE.ipynb)

# Taxonomy Enrichment without candidates

In [1]:
!git clone https://github.com/palette-knife25/candidate-free-te

fatal: destination path 'candidate-free-te' already exists and is not an empty directory.


In [2]:
!pip install pytorch-lightning
!pip install transformers

In [3]:
!pip install hydra-core --upgrade

Requirement already up-to-date: hydra-core in /usr/local/lib/python3.7/dist-packages (1.0.6)


In [1]:
import os
import sys
import pathlib
src_dir = "/content/candidate-free-te"
sys.path.append(src_dir)

In [2]:
import hydra
from hydra.core.hydra_config import HydraConfig
from hydra.experimental import initialize, compose
from omegaconf import OmegaConf
import torch
import options

In [3]:
from train import train
from evaluate import evaluate

In [4]:
def prepare_config(overrides):
    os.chdir(src_dir)
    with initialize("configs"):
        cfg = compose(config_name='config.yaml',
                      overrides=overrides, 
                      return_hydra_config=True,)

    # This is needed because in compose api, hydra is not initialized properly
    pathlib.Path(cfg.hydra.run.dir).mkdir(parents=True, exist_ok=True) 
    os.chdir(cfg.hydra.run.dir)
    return cfg

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/candidate-free-te/experiments

In [10]:
overrides = [
  "experiment=fixed",
  "experiment.name=fixed_twitter200",
  "+experiment.dataset.tokenizer.w2v_model='glove-twitter-200'",
  "experiment.trainer_args.max_epochs=30",
  "experiment.scheduler.step_size=10",
  f"experiment.trainer_args.gpus={torch.cuda.device_count()}",
  "experiment.evaluate_dl=test_dataloader"
]

In [11]:
def best_checkpoint_path(cfg):
  ckpts = os.listdir(os.path.join(src_dir, cfg.hydra.run.dir, "checkpoints"))
  best = [ckpt for ckpt in ckpts if ckpt.startswith("best")][0]
  ckpt = os.path.join(src_dir, cfg.hydra.run.dir, "checkpoints", best)
  return ckpt

In [38]:
overrides[1] = "experiment.name=fixed_twitter200"
overrides[2] = "+experiment.dataset.tokenizer.w2v_model='glove-twitter-200'"

cfg = prepare_config(overrides)
train(cfg.experiment)

ckpt_path = best_checkpoint_path(cfg)
overrides.append(f"+experiment.trainer_args.resume_from_checkpoint='{ckpt_path}'")
cfg = prepare_config(overrides)
evaluate(cfg.experiment)
overrides = overrides[:-1]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores


Loading the training data


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type           | Params
---------------------------------------------
0 | tokenizer | FixedTokenizer | 238 M 
1 | net       | FixedEnricher  | 242 M 
---------------------------------------------
4.0 M     Trainable params
238 M     Non-trainable params
242 M     Total params
970.710   Total estimated model params size (MB)



Loading the training data
Loading the test data


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'ValidationMAP': 0.0016913579311221838,
 'ValidationMRR': 0.0016913579311221838,
 'ValidationPrecision@10': 0.00023703706392552704}
--------------------------------------------------------------------------------


In [ ]:
overrides[1] = "experiment.name=fixed_fasttext300"
overrides[2] = "+experiment.dataset.tokenizer.w2v_model='fasttext-wiki-news-subwords-300'"

cfg = prepare_config(overrides)
train(cfg.experiment)

ckpt_path = best_checkpoint_path(cfg)
overrides.append(f"+experiment.trainer_args.resume_from_checkpoint='{ckpt_path}'")
cfg = prepare_config(overrides)
evaluate(cfg.experiment)
overrides = overrides[:-1]

In [ ]:
overrides[1] = "experiment.name=fixed_gwiki300"
overrides[2] = "+experiment.dataset.tokenizer.w2v_model='glove-wiki-gigaword-300'"

cfg = prepare_config(overrides)
train(cfg.experiment)

ckpt_path = best_checkpoint_path(cfg)
overrides.append(f"+experiment.trainer_args.resume_from_checkpoint='{ckpt_path}'")
cfg = prepare_config(overrides)
evaluate(cfg.experiment)
overrides = overrides[:-1]

In [ ]:
import gensim.downloader as api
import os
import json

def read_json(path: str):
  with open(path, "r") as f:
    return json.load(f)

def get_vocab(pretrained):
  w2v = api.load(pretrained)
  return set(w2v.vocab.keys())

In [ ]:
pre_list = ['fasttext-wiki-news-subwords-300',
            'word2vec-google-news-300',
            'glove-wiki-gigaword-50',
            'glove-twitter-25']

In [ ]:
def check_common(stage="train"):
  json_path = f"/content/candidate-free-te/datasets/newset/json/{stage}_graph_relations_bert_3K_rel.json"
  json = read_json(json_path)
  targets = []
  for synset in json.values():
    targets.extend([lemma.lower().replace('_', '-') for lemma in synset['lemmas']])
  target_set = set(targets)

  print(f"For {stage} set")
  for name in pre_list:
    vocab = get_vocab(name)
    print(f"{name} -> {len(vocab)} total words ")
    common = vocab.intersection(target_set)
    print(f"Common {len(common)} / {len(target_set)} = {len(common)/len(target_set)}")

In [ ]:
check_common("train")

For train set
fasttext-wiki-news-subwords-300 -> 999999 total words 
Common 38560 / 100937 = 0.38202046821284563
word2vec-google-news-300 -> 3000000 total words 
Common 30240 / 100937 = 0.2995928153204474
glove-wiki-gigaword-50 -> 400000 total words 
Common 34170 / 100937 = 0.338527992708323
glove-twitter-25 -> 1193514 total words 
Common 23805 / 100937 = 0.23584017753648315


In [ ]:
check_common("test")

For test set
fasttext-wiki-news-subwords-300 -> 999999 total words 
Common 3231 / 5226 = 0.6182548794489093
word2vec-google-news-300 -> 3000000 total words 
Common 2724 / 5226 = 0.521239954075775
glove-wiki-gigaword-50 -> 400000 total words 
Common 2880 / 5226 = 0.5510907003444316
glove-twitter-25 -> 1193514 total words 
Common 2176 / 5226 = 0.41637964026023727
